In [1]:
import duckdb
from typing import Any
import json
import geopandas as gpd
from osgeo import ogr

In [2]:
con = duckdb.connect()
print(con.execute('SELECT 42').fetchall())

[(42,)]


In [13]:
# To install an extension (this is usually done only once)
try:
    con.install_extension('httpfs')
    con.install_extension('spatial')
    con.load_extension('httpfs')
    con.load_extension('spatial')
    con.execute("SET s3_region='us-west-2'")
except Exception as e:
    print(f"Failed to install extension: {e}")

In [15]:


# Set S3 region
con.execute("SET s3_region='us-west-2'")

# Define the query to read from S3 and filter the data
query = '''
SELECT
    type,
    subType,
    localityType,
    adminLevel,
    isoCountryCodeAlpha2,
    JSON(names) AS names,
    JSON(sources) AS sources,
    ST_GeomFromWkb(geometry) AS geometry
FROM read_parquet(
    's3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=admins/type=*/*',
    filename=true,
    hive_partitioning=1
)
WHERE adminLevel = 2
    AND ST_GeometryType(ST_GeomFromWkb(geometry)) IN ('POLYGON','MULTIPOLYGON')
'''

# Execute the query and store the result
df = con.execute(query).fetchdf()

# Convert the WKB geometry to Shapely geometry objects
df['geometry'] = df['geometry'].apply(wkb.loads)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Convert GeoDataFrame to GeoJSON
geojson_data = json.loads(gdf.to_json())

# Write the GeoJSON data to a file
with open('countries.geojson', 'w') as f:
    json.dump(geojson_data, f)

# Close the DuckDB connection
con.close()


NameError: name 'wkb' is not defined